# E.T.L. Completo - <span style = "color:green"> Esperanza de Vida al Nacer </span>

In [1]:
# Carga de módulos necesarios
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import wbgapi as wb
import pandas as pd
import numpy as np

import dtale
import dtale.app as dtale_app
import missingno as msno

pd.options.display.max_columns = None

In [2]:
# Crear df desde API Banco Mundial

def crerate_df_bm(indicators, countries, db=2):
    '''
    Construir Data Frame con información del World Bank.

     Args:
         indicators: Lista de diccionarios con los códigos y nombres de indicadores
         countries: Lista con los códigos de los paises según ISO 3166-1
         db: Código base de datos a rastrear por defecto db=2
       
     Returns:
         Data Frame 
    '''
    
    # Crear array vacio
    data = np.array([])
    # Buscar a través de la lista de indicadores y obtener los datos
    for i in indicators.items():
        trans_data = wb.data.fetch(i[0], countries, db=db)
        list_countries = []
        list_years = []
        list_data = []
        # Se agregan los valores obtenidos a cada lista auxiliar y se crea diccionario con estas
        for x in trans_data:
            list_countries.append(x['economy'])
            list_years.append((x['time'][2::]))
            list_data.append((x['value']))
        data_dict = {'name':i[1],
                'country':list_countries,
                'year':list_years,
                'value':list_data
              }
        # Se Agregan data_dict al array data
        data = np.append(data, data_dict)
        
    # Construcción del data frame    
    df = pd.DataFrame()
    df['country'] = data[0]['country'] 
    df['year'] = data[0]['year'] 
    for i in range(0,len(data)):
        df[data[i]['name']] = data[i]['value']
        
    return df

In [3]:
# Generar df paraPaíses a estudiar

countries_list = ['CHN', 'RUS', 'PHL', 'IND', 'KOR', 'ZAF', 
                  'ARG', 'AUS', 'BRA', 'BOL', 'CHL', 'VEN',
                  'ECU', 'MEX', 'PER', 'SLV', 'URY', 'PRY',
                  'ESP', 'SWE', 'CHE', 'GBR', 'PRT', 'JPN', 
                  'ITA', 'DEU', 'FRA', 'NOR', 'LUX',
                  'CAN', 'COL', 'USA', 'EGY'
                 ]
# Crear listado de Países (economy) que se encuentran en la Base de Datos
countries = wb.economy.list()
countries_df = pd.DataFrame(countries)
# Generar Data Frame de los 34 paises a estudiar
countries_df = countries_df[countries_df['id'].isin(countries_list)][['id', 'value', 'longitude', 'latitude', 'region', 'lendingType', 'incomeLevel']]
countries_df

,id,value,longitude,latitude,region,lendingType,incomeLevel
9,ARG,Argentina,-58.417300,-34.611800,LCN,IBD,UMC
13,AUS,Australia,149.129000,-35.282000,EAS,LNX,HIC
28,BOL,Bolivia,-66.193600,-13.990800,LCN,IBD,LMC
29,BRA,Brazil,-47.929200,-15.780100,LCN,IBD,UMC
35,CAN,Canada,-75.691900,45.421500,NAC,LNX,HIC
37,CHE,Switzerland,7.448210,46.948000,ECS,LNX,HIC
39,CHL,Chile,-70.647500,-33.475000,LCN,IBD,HIC
40,CHN,China,116.286000,40.049500,EAS,IBD,UMC
45,COL,Colombia,-74.082000,4.609870,LCN,IBD,UMC
55,DEU,Germany,13.411500,52.523500,ECS,LNX,HIC


## Descripción de columnas data frame
#### id
Código País según ISO 3166-1 alfa-3.
#### value
Nombre común del país o territorio comúnmente usado.
#### longitude
Medida que en cartografía expresa la distancia angular entre un punto dado de la superficie terrestre y el meridiano que se toma como 0°.
#### latitude
Distancia angular entre la línea ecuatorial (el ecuador) y un punto determinado de la Tierra, medida a lo largo del meridiano en el que se encuentra dicho punto.
#### region
Código de la región geográfica en la que se encuentra el país o territorio.
#### Lending Type
Categoria que representa el riesgo respecto a prestamos<br> 
**IBD**: Préstamos respaldados por hipotecas (MBS) que se consideran de alto riesgo. Los MBS son valores que están respaldados por una cartera de hipotecas. Los MBS IBD suelen tener un mayor riesgo de incumplimiento que los MBS tradicionales debido a la calidad de las hipotecas subyacentes.<br>
**LNX**: Préstamos comerciales apalancados (LBO) que se consideran de alto riesgo. Los LBO son préstamos que se utilizan para financiar la adquisición de una empresa por parte de otra empresa. Los LBO suelen tener un mayor riesgo de incumplimiento que los préstamos comerciales tradicionales debido al alto nivel de apalancamiento involucrado.<br>
**IDX**: Préstamos estructurados que se consideran de alto riesgo. Los préstamos estructurados son préstamos que se han dividido en diferentes tramos de riesgo. Los tramos de mayor riesgo de los préstamos estructurados suelen tener un mayor riesgo de incumplimiento que los préstamos tradicionales.
#### Income Level
Código el tipo de ingresos del pais.
**HIC**: High income: Países de altos ingresos. El PIB per cápita de estos países es superior a 12.616 dólares.<br>
**UMC**: Upper Middle income: Países de ingresos medios altos. El PIB per cápita de estos países está entre 4.036 y 12.615 dólares.<br>
**LMC**: Lower Middle income: Países de ingresos medios bajos. El PIB per cápita de estos países está entre 1.036 y 4.035 dólares.<br>
**LIC**: Low income: Países de bajos ingresos. El PIB per cápita de estos países es inferior a 1.035 dólares.<br>
**INX**: Sin Información<br>

In [4]:
indicators =  {
                'SP.DYN.LE00.FE.IN': 'Life expectancy at birth, female (years)',
                'SP.DYN.LE00.IN': 'Life expectancy at birth, total (years)',
                'SP.DYN.LE00.MA.IN': 'Life expectancy at birth, male (years)',
                'SP.URB.TOTL.IN.ZS': 'Urban population (% of total population)',
                'SP.RUR.TOTL.ZS': 'Rural population (% of total population)',
                'SP.POP.GROW':'Population growth (annual %)',
                'FP.CPI.TOTL.ZG': 'Inflation, consumer prices (annual %)',
                'NY.GDP.DEFL.KD.ZG': 'Inflation, GDP deflator (annual %)',
                'NY.GDP.MKTP.CD': 'GDP (current US$)',
                'NY.GDP.PCAP.CD': 'GDP per capita (current US$)',
                'NY.GDP.PCAP.KD.ZG': 'GDP per capita growth (annual %)',
                'NY.GNP.MKTP.CD': 'GNI (current US$)',
                'MS.MIL.XPND.GD.ZS': 'Military expenditure (% of GDP)',
                'NE.CON.GOVT.CD':'General government final consumption expenditure (current US$)',
                'TX.VAL.FOOD.ZS.UN':'Food exports (% of merchandise exports)',
                'AG.PRD.FOOD.XD':'Food production index (2014-2016 = 100)',
                'NE.CON.PRVT.ZS': 'Households and NPISHs final consumption expenditure (% of GDP)',
                'SP.DYN.AMRT.FE': 'Mortality rate, adult, female (per 1,000 female adults)',
                'SP.DYN.AMRT.MA': 'Mortality rate, adult, male (per 1,000 male adults)',
                'SP.DYN.IMRT.FE.IN': 'Mortality rate, infant, female (per 1,000 live births)',
                'SP.DYN.IMRT.IN': 'Mortality rate, infant (per 1,000 live births)',
                'SP.DYN.IMRT.MA.IN': 'Mortality rate, infant, male (per 1,000 live births',
                'SP.DYN.CBRT.IN': 'Birth rate, crude (per 1,000 people)',
                'NY.GDY.TOTL.KN': 'Gross domestic income (constant LCU)',
               }

indicator25 = {
        'SL.EMP.MPYR.FE.ZS': 'Employers, female (% of female employment) (modeled ILO estimate)',
        'SL.EMP.MPYR.MA.ZS': 'Employers, male (% of male employment) (modeled ILO estimate)',
        'SL.EMP.MPYR.ZS': 'Employers, total (% of total employment) (modeled ILO estimate)',
        'SL.EMP.OWAC.ZS': 'Own-account workers, total (% of male employment) (modeled ILO estimate)',
        'SL.EMP.SELF.ZS': 'Self-employed, total (% of total employment) (modeled ILO estimate)',
        'SL.EMP.TOTL': 'Total employment, total (ages 15+)',
}

countries = countries_df['id'].tolist()

### Data Frame 1: Indicadores Base de Datos `World Development Indicators`

In [5]:
df = crerate_df_bm(indicators, countries)
df 

,country,year,"Life expectancy at birth, female (years)","Life expectancy at birth, total (years)","Life expectancy at birth, male (years)",Urban population (% of total population),Rural population (% of total population),Population growth (annual %),"Inflation, consumer prices (annual %)","Inflation, GDP deflator (annual %)",GDP (current US$),GDP per capita (current US$),GDP per capita growth (annual %),GNI (current US$),Military expenditure (% of GDP),General government final consumption expenditure (current US$),Food exports (% of merchandise exports),Food production index (2014-2016 = 100),Households and NPISHs final consumption expenditure (% of GDP),"Mortality rate, adult, female (per 1,000 female adults)","Mortality rate, adult, male (per 1,000 male adults)","Mortality rate, infant, female (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)","Mortality rate, infant, male (per 1,000 live births","Birth rate, crude (per 1,000 people)",Gross domestic income (constant LCU)
0,ZAF,2022,NaN,NaN,NaN,68.335,31.665,0.841058,7.039727,5.054293,4.058697e+11,6776.480077,1.187663,3.974388e+11,0.739894,7.949644e+10,10.392224,NaN,63.094095,NaN,NaN,NaN,NaN,NaN,NaN,4.748844e+12
1,ZAF,2021,64.999,62.341,59.458,67.847,32.153,0.998920,4.611672,6.219017,4.190156e+11,7055.055176,3.870315,4.109907e+11,0.809813,8.232470e+10,9.657360,114.31,61.674318,294.817,399.441,24.4,26.4,28.4,19.821,4.784044e+12
2,ZAF,2020,67.964,65.252,62.178,67.354,32.646,1.223179,3.210036,5.692615,3.376196e+11,5741.641249,-7.481093,3.319875e+11,0.957517,7.019953e+10,11.846560,111.39,62.524709,256.616,353.927,24.8,26.9,28.9,20.326,4.492816e+12
3,ZAF,2019,69.107,66.175,62.834,66.856,33.144,1.295074,4.120246,4.638081,3.885312e+11,6688.774746,-0.987175,3.788474e+11,0.884245,7.644400e+10,10.975456,104.54,63.931415,236.101,341.836,25.2,27.3,29.3,21.276,4.687792e+12
4,ZAF,2018,68.740,65.674,62.203,66.355,33.645,1.225530,4.517165,3.745754,4.041589e+11,7048.508112,0.285736,3.928701e+11,0.897196,7.845114e+10,11.109532,104.32,63.903014,246.989,360.378,25.7,27.8,29.8,21.137,4.622462e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2074,ARG,1964,68.261,64.363,61.037,75.844,24.156,1.599523,NaN,28.774617,2.560525e+10,1179.504098,8.382751,2.626786e+10,1.150000,1.851010e+09,77.226652,33.65,68.671663,156.218,270.349,NaN,NaN,NaN,23.437,2.020451e+11
2075,ARG,1963,68.285,64.449,61.178,75.309,24.691,1.621661,NaN,25.591154,1.827212e+10,855.275656,-6.831393,1.874495e+10,1.680000,1.873294e+09,72.124154,36.20,71.761256,155.531,266.354,NaN,NaN,NaN,23.663,1.798748e+11
2076,ARG,1962,68.031,64.244,61.035,74.767,25.233,1.629282,NaN,28.871842,2.445060e+10,1163.186836,-2.454333,2.508341e+10,2.660000,2.981109e+09,71.516722,33.20,67.053969,158.609,268.467,NaN,NaN,NaN,23.798,1.853979e+11
2077,ARG,1961,68.031,64.360,61.240,74.217,25.783,1.613029,NaN,20.310698,NaN,NaN,3.740903,NaN,2.140000,NaN,NaN,31.41,62.311085,158.440,263.573,NaN,NaN,NaN,23.638,2.001264e+11


In [6]:
# Cambiar tipo de datos de columna "year"
df['year'] = df['year'].astype('int')
# Eliminar filas donde el año (year) sea anterior a 1990
df.drop(df[(df['year'] < 1990)].index, inplace=True)
# Valores Nulos
df.isnull().sum()*100/df.shape[0]

country                                                           0.000000
year                                                              0.000000
Life expectancy at birth, female (years)                          3.030303
Life expectancy at birth, total (years)                           3.030303
Life expectancy at birth, male (years)                            3.030303
Urban population (% of total population)                          0.000000
Rural population (% of total population)                          0.000000
Population growth (annual %)                                      0.000000
Inflation, consumer prices (annual %)                             5.693297
Inflation, GDP deflator (annual %)                                0.734619
GDP (current US$)                                                 0.734619
GDP per capita (current US$)                                      0.734619
GDP per capita growth (annual %)                                  0.734619
GNI (current US$)        

### Data Frame 2: Indicadores Base de Datos `Jobs`

In [7]:
df2 = crerate_df_bm(indicator25, countries, db=25)
df2

,country,year,"Employers, female (% of female employment) (modeled ILO estimate)","Employers, male (% of male employment) (modeled ILO estimate)","Employers, total (% of total employment) (modeled ILO estimate)","Own-account workers, total (% of male employment) (modeled ILO estimate)","Self-employed, total (% of total employment) (modeled ILO estimate)","Total employment, total (ages 15+)"
0,ZAF,2016,2.652,7.760,5.540,9.113000,15.151000,15940018.0
1,ZAF,2015,2.268,7.419,5.175,8.872000,14.619000,15977546.0
2,ZAF,2014,2.513,7.491,5.321,8.277000,14.162000,15415824.0
3,ZAF,2013,2.591,7.502,5.358,8.897000,14.899000,15159193.0
4,ZAF,2012,2.447,7.546,5.354,9.436000,15.457000,14710100.0
...,...,...,...,...,...,...,...,...
886,ARG,1994,2.429,6.211,4.734,23.333000,29.740999,13003003.0
887,ARG,1993,3.174,7.411,5.773,24.106001,31.165001,12984138.0
888,ARG,1992,2.411,7.324,5.394,23.100000,29.774000,13230241.0
889,ARG,1991,2.016,6.615,4.828,23.792999,29.705000,13021217.0


In [8]:
# Cambiar tipo de datos de columna "year"
df2['year'] = df2['year'].astype('int')
# Eliminar filas donde el año (year) sea anterior a 1990
df2.drop(df2[(df2['year'] < 1990)].index, inplace=True)
# Valores Nulos
df2.isnull().sum()*100/df2.shape[0]

country                                                                     0.000000
year                                                                        0.000000
Employers, female (% of female employment) (modeled ILO estimate)           3.703704
Employers, male (% of male employment) (modeled ILO estimate)               3.703704
Employers, total (% of total employment) (modeled ILO estimate)             3.703704
Own-account workers, total (% of male employment) (modeled ILO estimate)    3.703704
Self-employed, total (% of total employment) (modeled ILO estimate)         3.703704
Total employment, total (ages 15+)                                          3.703704
dtype: float64

### Data Frame 3: Indicador de Internet

In [9]:
df3 = pd.read_csv('datasets/internet.csv')
df3 = df3.rename(columns={'Code':'country', 'Year':'year'})
df3 = df3.drop(columns=['Unnamed: 0','Entity','Cellular Subscription','No. of Internet Users', 'Broadband Subscription'])

In [10]:
df3

,country,year,Internet Users(%)
0,AFG,1980,0.000000
1,AFG,1981,0.000000
2,AFG,1982,0.000000
3,AFG,1983,0.000000
4,AFG,1984,0.000000
...,...,...,...
8862,ZWE,2016,23.119989
8863,ZWE,2017,24.400000
8864,ZWE,2018,25.000000
8865,ZWE,2019,25.100000


### Data New Frame: Crear data Frame de Trabajo con el Merge de `df`, `df2` y `df3`

In [11]:
new_df = df.merge(df2, how='left', on=['country', 'year'])
new_df = new_df.merge(df3, how='left', on=['country', 'year'])
new_df.isnull().sum()*100/new_df.shape[0]

country                                                                      0.000000
year                                                                         0.000000
Life expectancy at birth, female (years)                                     3.030303
Life expectancy at birth, total (years)                                      3.030303
Life expectancy at birth, male (years)                                       3.030303
Urban population (% of total population)                                     0.000000
Rural population (% of total population)                                     0.000000
Population growth (annual %)                                                 0.000000
Inflation, consumer prices (annual %)                                        5.693297
Inflation, GDP deflator (annual %)                                           0.734619
GDP (current US$)                                                            0.734619
GDP per capita (current US$)                          

### Evaluación del data Frame EDA

In [12]:
dtale.show(df, host='localhost')

2023-10-12 10:21:00,007 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,217 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,220 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,223 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,225 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,228 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,230 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,251 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,253 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,255 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,257 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,259 - WARNING  - findfont: Font family 'Heiti TC' not found.
2023-10-12 10:21:00,261 - WA

## <span style="color:green">Fin de E.T.L.</span>